In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os
os.chdir('/content/drive/MyDrive/files/datasets/data')
!ls

test  train  valid


In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator


In [ ]:
tf.__version__

'2.7.0'

In [ ]:
train_path = '/content/drive/MyDrive/files/datasets/data/train'
test_path = '/content/drive/MyDrive/files/datasets/data/test'
valid_path = '/content/drive/MyDrive/files/datasets/data/valid'

## Part 1 Data Preprocessing
1. preprocessing training set
2. preprocessing test set

In [ ]:
train_datagen = ImageDataGenerator(
    rescale = 1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

training_set = train_datagen.flow_from_directory(
    train_path, 
    target_size=(64,64),
    batch_size=32,
    class_mode='binary'
)

Found 1000 images belonging to 2 classes.


In [ ]:
test_datagen = ImageDataGenerator(
    rescale = 1./255
)             # test_datagen does not need transformation to avoid leakage

test_set = test_datagen.flow_from_directory(
    test_path,
    target_size=(64,64),
    batch_size=32,
    class_mode='binary' 
)

Found 100 images belonging to 2 classes.


## build a cnn

In [ ]:
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, BatchNormalization, Conv2D, MaxPool2D


In [ ]:
# cnn = Sequential([
#             Conv2D(filters=32, kernel_size=(3,3), activation='relu', input_shape=(64,64,3)),
#             MaxPool2D(pool_size=(2,2), strides=2),
#             Conv2D(filters=64, kernel_size=(3,3), activation='relu'),
#             MaxPool2D(pool_size=64, strides=2),
#             Flatten(),
#             Dense(units=2, activation='softmax')
# ])

cnn = Sequential([
    Conv2D(filters=32, kernel_size=(3, 3), activation='relu',
           padding='same', input_shape=(64, 64, 3)),
    MaxPool2D(pool_size=(2, 2), strides=2),
    Conv2D(filters=64, kernel_size=(3, 3), activation='relu', padding='same'),
    MaxPool2D(pool_size=(2, 2), strides=2),
    Flatten(),
    Dense(units=128, activation='relu'),
    Dense(units=1, activation='sigmoid')
])


In [ ]:
cnn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 64, 64, 32)        896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 32, 32, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 32, 32, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 16, 16, 64)       0         
 2D)                                                             
                                                                 
 flatten (Flatten)           (None, 16384)             0         
                                                                 
 dense (Dense)               (None, 128)               2

In [ ]:
cnn.compile(
    optimizer='adam',
    loss = 'binary_crossentropy',
    metrics =['accuracy']
)

In [ ]:
cnn.fit(x=training_set, validation_data=test_set, epochs=25, verbose=2)

Epoch 1/25
32/32 - 226s - loss: 0.7405 - accuracy: 0.5380 - val_loss: 0.6963 - val_accuracy: 0.5000 - 226s/epoch - 7s/step
Epoch 2/25
32/32 - 6s - loss: 0.6838 - accuracy: 0.5450 - val_loss: 0.6939 - val_accuracy: 0.5600 - 6s/epoch - 196ms/step
Epoch 3/25
32/32 - 6s - loss: 0.6744 - accuracy: 0.5960 - val_loss: 0.6608 - val_accuracy: 0.6300 - 6s/epoch - 196ms/step
Epoch 4/25
32/32 - 6s - loss: 0.6369 - accuracy: 0.6540 - val_loss: 0.6380 - val_accuracy: 0.6800 - 6s/epoch - 196ms/step
Epoch 5/25
32/32 - 6s - loss: 0.6254 - accuracy: 0.6540 - val_loss: 0.5860 - val_accuracy: 0.7300 - 6s/epoch - 193ms/step
Epoch 6/25
32/32 - 6s - loss: 0.6096 - accuracy: 0.6440 - val_loss: 0.5908 - val_accuracy: 0.6900 - 6s/epoch - 196ms/step
Epoch 7/25
32/32 - 6s - loss: 0.5973 - accuracy: 0.6910 - val_loss: 0.5732 - val_accuracy: 0.6900 - 6s/epoch - 195ms/step
Epoch 8/25
32/32 - 6s - loss: 0.5487 - accuracy: 0.7300 - val_loss: 0.5675 - val_accuracy: 0.7200 - 6s/epoch - 194ms/step
Epoch 9/25
32/32 - 6s -

In [ ]:
import numpy as np
from tensorflow.keras.preprocessing import image

test_image = image.load_img('/content/drive/MyDrive/files/datasets/data/valid/dog/dog.10021.jpg', target_size=(64,64))
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis=0)

In [ ]:
test_image = test_image/255

In [ ]:
result = cnn.predict(test_image)
training_set.class_indices
if result[0][0] > 0.5:
  prediction = "dog"
else:
  prediction = 'cat'

print(prediction)
print(result)

dog
[[0.9998728]]


In [ ]:
training_set.class_indices

{'cat': 0, 'dog': 1}